In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

!ls

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
gdrive	sample_data


In [1]:
import os

home_dir = '/content/gdrive/My Drive/ATFTransformer'
home_dir = '/hy-tmp'
os.chdir(home_dir)
!pwd

!pip install tqdm
from glob import glob
from tqdm import tqdm

import numpy as np
import pandas as pd

import multiprocessing
import time

from multiprocessing import Pool, freeze_support

/hy-tmp
Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 KB 217.3 kB/s eta 0:00:00a 0:00:01
You should consider upgrading via the '/usr/bin/python3.8 -m pip install --upgrade pip' command.


In [4]:
data_dir = os.path.join(home_dir)
os.chdir(data_dir)
!pwd
# !ls

files = glob(os.path.join(data_dir, '*.npz'))

print(files)

/hy-tmp
['/hy-tmp/corn_histogram_all_full.npz']


In [5]:
filename = '/hy-tmp/corn_histogram_all_full.npz'

hists = np.load(filename)

print(type(hists), list(hists.keys()))
print(type(hists['output_image']), hists['output_image'].shape)

<class 'numpy.lib.npyio.NpzFile'> ['output_image', 'output_yield', 'output_year', 'output_locations', 'output_index', 'output_areas']
<class 'numpy.ndarray'> (320, 9, 32, 512)


In [ ]:
freeze_support()

print(type(hists['output_image']), hists['output_image'].shape)

bins_name = list()
bins_valu = list()

for bin in range(0, hists['output_image'].shape[3]):
  bins_name.append(f'bin{bin}')

# print(bins_name)
print(hists['output_image'][0][0][0].shape)
# print(list(hists['output_image'][0][0][0]))
print(hists['output_index'][100][1])

# fn

idx_s = [x for x in range(0,319,29)]
idx_e = [x for x in range(29,320,29)]

print(len(idx_s), idx_s)
print(len(idx_e), idx_e)

# idx_s = [x for x in range(0,320,40)]
# idx_e = [x for x in range(40,321,40)]

# print(len(idx_s), idx_s)
# print(len(idx_e), idx_e)

# fn

def get_ds(idx_s, idx_e):
    hists = np.load('/hy-tmp/corn_histogram_all_full.npz')
    df = pd.DataFrame()

    time_idx = 0
    index_idx = 0
    for idx in range(idx_s, idx_e):
      for time_idx in range(0, hists['output_image'].shape[2]):
        for band_idx in range(0, hists['output_image'].shape[1]):
          bins_dict = dict(zip(bins_name,list(hists['output_image'][idx][band_idx][time_idx])))

          df1 = pd.DataFrame( hists['output_year'][idx], columns=['years'], index=[index_idx] )
          df2 = pd.DataFrame( hists['output_yield'][idx], columns=['yield'], index=[index_idx] )
          df3 = pd.DataFrame( hists['output_index'][idx][1], columns=['county'], index=[index_idx] )
          df4 = pd.DataFrame( f'band_{band_idx}', columns=['bands'], index=[index_idx] )
          df5 = pd.DataFrame( time_idx, columns=['time_idx'], index=[index_idx] )
          df6 = pd.DataFrame(bins_dict, index=[index_idx])

          df_r = pd.concat( [df1 , df2], axis=1)
          df_r = pd.concat( [df_r, df3], axis=1)
          df_r = pd.concat( [df_r, df4], axis=1)
          df_r = pd.concat( [df_r, df5], axis=1)
          df_r = pd.concat( [df_r, df6], axis=1)

          df = pd.concat( [df, df_r], axis=0)

          index_idx = index_idx + 1
            
    return df

with Pool(processes=11) as pool:
    results = pool.starmap(get_ds, zip(idx_s, idx_e))

# df.to_csv('china_prl.csv')


# fn
# for indx in range(0, hists.shape[0]):
#   for time_idx in range(0, hists.shape[2]):
#     for bands in range(0, hists.shape[1]):


<class 'numpy.ndarray'> (320, 9, 32, 512)
(512,)
9
11 [0, 29, 58, 87, 116, 145, 174, 203, 232, 261, 290]
11 [29, 58, 87, 116, 145, 174, 203, 232, 261, 290, 319]


In [6]:
print(type(results), len(results))

filesave = 'corn_china_pandas.csv'

df = pd.DataFrame()
for ii in range(0, 11):
    df = pd.concat( [df, results[ii]], axis=0 )
    
df.head()
    
df.to_csv(filesave)
    

<class 'list'> 11


In [11]:
ds = pd.read_csv(filesave)
ds[8340:8390]

,Unnamed: 0,years,yield,county,bands,time_idx,bin0,bin1,bin2,bin3,...,bin502,bin503,bin504,bin505,bin506,bin507,bin508,bin509,bin510,bin511
9825,1473,2005,0.588182,2,band_6,3,0.000067,0.000077,0.000100,0.000085,...,0.000042,0.000031,0.000021,0.000023,0.000027,0.000025,0.000040,0.000033,0.000035,0.000033
9826,1474,2005,0.588182,2,band_7,3,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9827,1475,2005,0.588182,2,band_8,3,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9828,1476,2005,0.588182,2,band_0,4,0.000000,0.000000,0.000000,0.000000,...,0.000102,0.000146,0.000154,0.000127,0.000133,0.000144,0.000121,0.000092,0.000137,0.000114
9829,1477,2005,0.588182,2,band_1,4,0.000023,0.000048,0.000027,0.000033,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9830,1478,2005,0.588182,2,band_2,4,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9831,1479,2005,0.588182,2,band_3,4,0.000000,0.000000,0.000000,0.000000,...,0.000002,0.000006,0.000002,0.000000,0.000006,0.000008,0.000004,0.000000,0.000002,0.000004
9832,1480,2005,0.588182,2,band_4,4,0.000015,0.000029,0.000025,0.000025,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9833,1481,2005,0.588182,2,band_5,4,0.000027,0.000010,0.000023,0.000017,...,0.000039,0.000031,0.000052,0.000044,0.000035,0.000037,0.000025,0.000033,0.000031,0.000044
9834,1482,2005,0.588182,2,band_6,4,0.000019,0.000029,0.000019,0.000012,...,0.000031,0.000044,0.000042,0.000060,0.000037,0.000025,0.000040,0.000025,0.000033,0.000056


In [13]:
ds

,Unnamed: 0,years,yield,county,bands,time_idx,bin0,bin1,bin2,bin3,...,bin502,bin503,bin504,bin505,bin506,bin507,bin508,bin509,bin510,bin511
0,0,2003,1.801242,0,band_0,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,1,2003,1.801242,0,band_1,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,2,2003,1.801242,0,band_2,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,3,2003,1.801242,0,band_3,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,4,2003,1.801242,0,band_4,0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91867,8347,2018,0.853096,29,band_4,31,0.000048,0.000145,0.000161,0.000048,...,0.000000,0.000000,0.000048,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
91868,8348,2018,0.853096,29,band_5,31,0.000048,0.000016,0.000048,0.000016,...,0.000048,0.000064,0.000016,0.000048,0.000032,0.000016,0.000064,0.000016,0.000032,0.0
91869,8349,2018,0.853096,29,band_6,31,0.000096,0.000080,0.000112,0.000016,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
91870,8350,2018,0.853096,29,band_7,31,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [8]:
!zip corn_china_pandas.csv.zip corn_china_pandas.csv
!oss cp corn_china_pandas.csv.zip oss://

  adding: china_prl.csv (deflated 75%)
Start at 2022-11-19 22:21:30.515317356 +0000 UTC


Parallel:      5                   Jobs:          5                   
Threshold:     50.00MB             PartSize:      auto                
VerifyLength:  false               VerifyMd5:     false               
CheckpointDir: /root/.hycloud_ossutil_checkpoint

[------------------------------] 100.00% 457.20KB/s 191.93MB/191.93MB 7m10.078s
Waiting for the uploaded key to be completed on server side.


Upload successfully, 191.93MB, n/a, /hy-tmp/china_prl.zip --> oss://china_prl.zip, cost [431121], status [200], request id [000001849202162490127BA143424FD0]


In [ ]:
os.system("shutdown /s /t 1")

In [ ]:
print(type(hists['output_image']), hists['output_image'].shape)
print(type(hists['output_yield']), hists['output_yield'].shape)
print(type(hists['output_year']), hists['output_year'].shape)
print(type(hists['output_locations']), hists['output_locations'].shape)
print(type(hists['output_index']), hists['output_index'].shape)
print(hists['output_year'])
# print(hists['output_index'])
# fn

In [ ]:
from matplotlib import pyplot as plt
# print(hists['output_index'])

bands = [x for x in range(0,9,1)]
years = [x for x in range(2003, 2019, 1)]
times = [x for x in range(0,32,1)]

print('years :', years)
      
counties = [0, 1, 2, 3, 5, 8, 9, 11, 12, 13, 14, 15, 16, 17, 19, 21, 23, 24, 26, 29]

num_bins = 512
bin_seq = np.linspace(1, num_bins, num_bins + 1) 

# fig, axs = plt.subplots(1, 9, figsize=(25,5))

it = 0
for county in range(0,len(counties),1):
  fig, axs = plt.subplots(1, 9, figsize=(25,5))
  for year in [0,1]:  #range(0,len(years),1): 
    for band in bands:
      for time in times:
        density = hists['output_image'][it,band,time,:]
        # print(len(density))
        axs[band].plot(bin_seq[0:-1], density)
      it = it + 1
    print(it)

  plt.show()
  